In [16]:
import numpy as np
import os

# WALK ON: GAIT RECOGNITION IN-THE-WILD

## Introduction

The widespreading of wearable and portable IoT devices is greatly improving quality of life thanks to the development of 5G and data analysis techniques. Every device we use can collect data regarding our daily activities and use it to help improve their practice. However this can also lead to great security and privacy vulnerabilities when the data handling procedure is not carried out correctly.

With the data collected from devices we already carry every day (like our smartphones), it is possible to collect gait data that can be used to identify activities or individuals.

## Objective

This project has the objetive to use gait data collected from different subject's phones to create and train a deep learning model that will be capable of identifying users given their gait information.

In particular data from the accelerometer and gyroscope will be used to detect signal sequences relative to a step. These will then be classified to identify different users depending on their walking patterns.

This problem can be divided in two smaller tasks i.e. extraction of the signal relative to a step (this can be done with standard signal processing techniques), and then classification using a neural network.

The former will be addressed with the procedure proposed in [1], which was developed for mobile phone sensors, but can easily be extended to any other wearable device. Classification, on the other hand, will be addressed with a simple CNN.

[1] Gadaleta, Matteo, and Michele Rossi. "Idnet: Smartphone-based gait recognition with convolutional neural networks." Pattern Recognition 74 (2018): 25-37.

## Dataset acquisition

The datasets used for the exploration and development of this project are given by [2]. There are four datasets dedicated to the goals this project wants to achieve, so every one of them will be used in order to more accurately measure the results obtained by the model developed.

[2] Zou Q, Wang Y, Zhao Y, Wang Q and Li Q, Deep learning-based gait recogntion using smartphones in the wild, IEEE Transactions on Information Forensics and Security, vol. 15, no. 1, pp. 3197-3212, 2020.

In [1]:
!mkdir datasets

### Utilities

Before the acquisition, reading and conversion of any dataset, a set of utility functions were set to better auxiliate in the reading and conversion process.

In [20]:
def load_gait_information(x: str, y: str, z: str):
    '''
    Read all the x, y and z values of a dataset of a given device readings.

    Args:
        x (str): The path for the dataset of x readings.
        y (str): The path for the dataset of y readings.
        z (str): The path for the dataset of z readings.

    Returns:
        np.array: The 3 * Steps * 128 matrix of the read information.
    '''

    # Create the final matrix where the readings will be stored
    signals = []

    # Iterative process for every axis reading
    for path in [x, y, z]:

        # Create a temporary matrix for storing a single axis readings
        axis = []

        print(path)

        # Open the dataset file location
        file = open(path, 'r')

        # Read every row of the axis readings
        for row in file:

            # Conversion of the row values and addition to the axis matrix
            axis.append(np.array(row.strip().split(' '), dtype=np.float32))

        # Addition of the axis matrix to the readings matrix
        signals.append(axis)

        # Closing of the read file
        file.close()

    # Returning and conversion of the final readings matrix
    return np.array(signals)

print(os.listdir('.'))
final = load_gait_information('./datasets/dataset1/train_acc_x.txt','datasets/dataset1/train/Inertial\ \Signals/train_acc_y.txt','datasets/dataset1/train/Inertial\ \Signals/train_acc_z.txt')
print(len(final))
print(final)

['README.md', 'utils.py', '.gitignore', '.git', 'gait-recognition.ipynb', 'datasets']
./datasets/dataset1/train_acc_x.txt
datasets/dataset1/train/Inertial\ \Signals/train_acc_y.txt


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/dataset1/train/Inertial\\ \\Signals/train_acc_y.txt'

### Dataset #1 - 118 Subjects, Interpolation

In [ ]:
!wget 'https://drive.usercontent.google.com/download?id=1yPNWWeubU9SjRKqW7H4-q0BaNsw57jWc&export=download&authuser=0&confirm=t&uuid=f7baf6b2-4139-40d1-b1c1-9ff21f5d696d&at=AIrpjvMKaZOBaOKN_juwrHVVepG3%3A1737392898709' -O datasets/dataset1.zip

!unzip datasets/dataset1.zip -d datasets/dataset1

!find datasets/dataset1 -mindepth 2 -type f -exec mv -t datasets/dataset1 {} +

!find datasets/dataset1 -type d -mindepth 1 -exec rm -r {} +

Archive:  datasets/dataset1.zip
   creating: datasets/dataset1/Dataset #1/
   creating: datasets/dataset1/Dataset #1/test/
   creating: datasets/dataset1/Dataset #1/test/Inertial Signals/
  inflating: datasets/dataset1/Dataset #1/test/Inertial Signals/test_acc_x.txt  
  inflating: datasets/dataset1/Dataset #1/test/Inertial Signals/test_acc_y.txt  
  inflating: datasets/dataset1/Dataset #1/test/Inertial Signals/test_acc_z.txt  
  inflating: datasets/dataset1/Dataset #1/test/Inertial Signals/test_gyr_x.txt  
  inflating: datasets/dataset1/Dataset #1/test/Inertial Signals/test_gyr_y.txt  
  inflating: datasets/dataset1/Dataset #1/test/Inertial Signals/test_gyr_z.txt  
  inflating: datasets/dataset1/Dataset #1/test/y_test.txt  
   creating: datasets/dataset1/Dataset #1/train/
   creating: datasets/dataset1/Dataset #1/train/Inertial Signals/
  inflating: datasets/dataset1/Dataset #1/train/Inertial Signals/train_acc_x.txt  
  inflating: datasets/dataset1/Dataset #1/train/Inertial Signals/tra

### Dataset #2 - 20 Subjects, Interpolation

### Dataset #3 - 118 Subjects, Time-fixed

### Dataset #4 - 20 Subjects, Time-fixed